<a href="https://colab.research.google.com/github/Ganesh039/Fmml_assignment/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [29]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [30]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-30-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [31]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [32]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [33]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [34]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [35]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [36]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [37]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [38]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [39]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [40]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [41]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [42]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


#**1.Does averaging the validation accuracy across multiple splits give more consistent results?**

Yes, averaging the validation accuracy across multiple splits of your dataset can often provide more consistent and reliable results compared to relying on a single split. This practice is commonly referred to as "cross-validation."

Cross-validation involves splitting your dataset into multiple subsets (folds), training and testing your model on different combinations of these subsets, and then averaging the performance metrics (such as accuracy) across the folds. The most common form of cross-validation is k-fold cross-validation, where the dataset is divided into k subsets, and the training and testing process is repeated k times, with each fold serving as the test set exactly once.

Here are some benefits of using cross-validation:

1. **Reduced Variance:**

  Cross-validation helps to reduce the variance in your model evaluation. Since different splits of the data may lead to variations in the validation accuracy, averaging these results provides a more stable estimate of your model's performance.

2. **Better Generalization Assessment:**   

  By testing your model on multiple different subsets of data, you get a better sense of how well your model generalizes to unseen data. This is particularly important because a single random split may result in a dataset that doesn't represent the underlying distribution of your data accurately.

3. **Maximizing Data Utilization:**

  Cross-validation allows you to use your entire dataset for both training and testing, which can be especially beneficial when your dataset is limited in size.

4. **Identifying Overfitting:**

 Cross-validation can help you detect overfitting. If you observe a significant difference between training and validation performance across the folds, it may indicate that your model is overfitting the data.

However, it's essential to note that cross-validation can be computationally expensive, especially with large datasets and complex models. It may not always be practical for every situation. Additionally, the choice of the number of folds (k) can impact the trade-off between computational cost and result stability.

In summary, cross-validation is a valuable technique for assessing the performance and reliability of machine learning models, as it provides more consistent results compared to a single split of the data. It's particularly useful when you have a limited amount of data and want a robust estimate of your model's performance.

#**2.Does it give more accurate estimate of test accuracy?**

  Cross-validation, while providing a more stable and reliable estimate of a model's performance compared to a single train-test split, doesn't necessarily give a more accurate estimate of the test accuracy you would expect to see on completely unseen data. Here's why:

1. **In-Sample vs. Out-of-Sample Performance:**

 When you perform cross-validation, you are still evaluating your model's performance on data from the same dataset it was trained on. This is sometimes referred to as "in-sample" performance. It provides an estimate of how well your model performs on different subsets of the training data, but it doesn't directly tell you how well your model will generalize to completely new, unseen data.

2. **Data Leakage:**

Cross-validation mitigates the risk of data leakage to some extent, but it doesn't eliminate it entirely. If there are any data leakage issues in your dataset (e.g., data preprocessing steps that use information from the entire dataset), cross-validation may not completely address this, and your performance estimates could still be overly optimistic.

3. **Dataset Bias:**

 Cross-validation assumes that the data is exchangeable, meaning that the data distribution is consistent across all folds. If your dataset has a temporal or spatial bias that cross-validation doesn't capture (e.g., time series data with trends), the cross-validated estimate may not accurately reflect real-world performance.

4. **Limited Variability:**

  Cross-validation provides an estimate of the model's average performance across different splits of the data. It's valuable for assessing the model's stability and identifying issues like overfitting, but it may not capture the full range of variability you might encounter when deploying the model in the real world.

To get a more accurate estimate of how your model will perform on completely new, unseen data, you should reserve a separate, independent test dataset that has not been used during model development or hyperparameter tuning. This test dataset should be kept entirely separate until you are ready to evaluate the final model. The performance on this test dataset provides a better estimate of how your model will perform in real-world scenarios.

In summary, while cross-validation provides valuable insights and a more stable estimate of your model's performance, it's not a substitute for a dedicated test dataset to assess the accuracy of your model on truly unseen data. Both techniques have their roles in the machine learning evaluation process.

#**3.What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?**


In machine learning, the number of iterations or epochs can have a significant effect on the training process and the resulting model's performance. However, it's important to clarify that the number of iterations typically refers to the number of times the model goes through the entire training dataset during the training process, and it is more relevant for training deep learning models and certain iterative optimization algorithms like gradient descent. It doesn't directly affect the estimate of test accuracy but can impact the model's ability to learn and generalize. Let's break down the effects:

1. **Underfitting and Overfitting:**
   - **Underfitting:** If you use too few iterations, your model may not have sufficient opportunities to learn from the data. It might result in underfitting, where the model fails to capture the underlying patterns in the data, leading to poor generalization.
   - **Overfitting:** Conversely, if you use too many iterations, especially with a complex model, your model might memorize the training data instead of learning to generalize from it. This can lead to overfitting, where the model performs well on the training data but poorly on unseen data.

2. **Convergence:**
   - The number of iterations also affects whether your model converges to a solution. In iterative optimization algorithms like gradient descent, the algorithm updates the model's parameters in each iteration to minimize the loss function. If you stop training too early, your model may not have converged to an optimal solution, and you might not have an accurate representation of the model's performance.

3. **Training Time:**
   - The number of iterations impacts the training time. More iterations generally require more time and computational resources. You need to strike a balance between training time and the quality of the model.

4. **Hyperparameter Tuning:**
   - The number of iterations can be considered a hyperparameter. In practice, you may need to experiment with different numbers of iterations during hyperparameter tuning to find the value that results in the best model performance on your validation or test data.

5. **Early Stopping:**
   - To address the risk of overfitting and control the number of iterations effectively, practitioners often use techniques like early stopping. Early stopping involves monitoring the model's performance on a validation dataset during training and stopping the training process when performance starts to degrade, indicating potential overfitting.

In summary, there isn't a one-size-fits-all answer to whether more iterations lead to a better estimate of test accuracy. The appropriate number of iterations depends on factors like the complexity of the model, the quality and quantity of the data, and the presence of early stopping mechanisms. You should aim to find a balance that allows your model to converge to an effective solution without overfitting or underfitting, and this may require experimentation and monitoring during the training process. Ultimately, the estimate of test accuracy should be assessed on a separate, independent test dataset to evaluate how well the model generalizes to unseen data.

#**4.Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?**

Increasing the number of iterations during training can sometimes help when dealing with a very small training dataset or validation dataset, but it's not a guaranteed solution and has its limitations. Here are some considerations:

**Advantages of Increasing Iterations:**

1. **Learning from Limited Data:**

 With a small dataset, the model may not have had the opportunity to learn all the underlying patterns and nuances in the data during a short training period. Increasing the number of iterations allows the model to see the same data multiple times, potentially improving its ability to learn from the limited data available.

2. **Fine-Tuning:**

 In some cases, especially with complex models like deep neural networks, additional iterations can allow the model to fine-tune its parameters, potentially leading to better convergence and improved performance.

**Limitations and Concerns:**

1. **Overfitting:**

Increasing iterations excessively on a small dataset can lead to overfitting. The model might start memorizing the training data rather than generalizing from it. You should monitor the validation performance to detect signs of overfitting and consider implementing early stopping to prevent it.

2. **Diminishing Returns:**

 There is a point of diminishing returns regarding the number of iterations. After a certain point, additional iterations may not significantly improve the model's performance, and they may increase the risk of overfitting or computational inefficiency.

3. **Data Quality:**

 The effectiveness of increasing iterations depends on the quality and diversity of your data. If your small dataset is not representative or lacks diversity, more iterations may not help much.

4. **Computational Cost:**

Training for a large number of iterations can be computationally expensive, especially with complex models. You should weigh the computational cost against the potential benefits.

**Alternative Strategies for Small Datasets:**

When dealing with very small datasets, you may want to consider alternative strategies in addition to increasing iterations:

1. **Data Augmentation:**

Augmenting your small dataset by creating variations of existing data samples (e.g., rotating, cropping, or adding noise) can artificially increase the effective size of your training dataset.

2. **Transfer Learning:**

Consider using transfer learning by starting with a pre-trained model and fine-tuning it on your small dataset. This can leverage knowledge learned from a larger dataset.

3. **Regularization:**

Apply regularization techniques like dropout, weight decay, or early stopping to mitigate overfitting when training on small datasets.

4. **Collect More Data:**

 If possible, consider collecting more data to increase the size of your dataset, which can lead to more robust and reliable model training.

In conclusion, while increasing the number of iterations during training can be one strategy to mitigate the challenges of small datasets, it should be employed cautiously and in conjunction with other techniques, and it may not always lead to the best results. The choice of strategy should depend on the specific characteristics of your dataset, the model architecture, and your computational resources.